In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr

In [ ]:
spark = SparkSession.builder.appName("example").getOrCreate()

In [ ]:
airports_file_path = "airport-codes-na.txt"
airports = spark.read.format("csv").options(header="true", inferSchema="true", sep="\t").load(airports_file_path)

In [ ]:
airports.createOrReplaceTempView("airports")

In [ ]:
departure_delays_file_path = "departuredelays.csv"
departure_delays = spark.read.format("csv").options(header="true").load(departure_delays_file_path)

In [ ]:
departure_delays = departure_delays\
  .withColumn("delay", expr("CAST(delay as INT) as delay"))\
  .withColumn("distance", expr("CAST(distance as INT) as distance"))

In [ ]:
departure_delays.createOrReplaceTempView("departure_delays")

In [ ]:
foo = departure_delays.filter(expr("""
origin == 'SEA' AND destination == 'SFO' AND date like '01010%' AND delay > 0
"""))

In [ ]:
foo.createOrReplaceTempView("foo")

In [ ]:
spark.sql("SELECT * FROM departure_delays LIMIT 10").show()

In [ ]:
spark.sql("SELECT * FROM foo").show()

In [ ]:
bar = departure_delays.union(foo)
bar.createOrReplaceTempView("bar")

In [ ]:
bar.filter(expr("origin == 'SEA' AND destination == 'SFO' AND date like '01010%' AND delay > 0")).show()

In [ ]:
foo.join(
  airports,
  airports.IATA == foo.origin
).select("City", "State", "date", "delay", "distance", "destination").show()

In [ ]:
departure_delays_sum = spark.sql("""
SELECT
  origin,
  destination,
  SUM(delay) as TotalDelays
FROM departure_delays
WHERE origin in ('SEA', 'SFO', 'JFK')
AND destination in ('SEA', 'SFO', 'JFK', 'DEN', 'ORD', 'LAX', 'ATL')
Group By origin, destination
""")

departure_delays_sum.createOrReplaceTempView("departure_delays_sum")

In [ ]:
spark.sql("""
SELECT origin, destination, TotalDelays, rank 
  FROM (
    SELECT
      origin,
      destination,
      TotalDelays,
      dense_rank() OVER (PARTITION BY origin ORDER BY TotalDelays DESC) as rank
    FROM departure_delays_sum
  ) t
  WHERE rank <= 3
""").show()

In [ ]:
foo2 = foo.withColumn(
    "status",
    expr("CASE WHEN delay <= 10 THEN 'On-time' ELSE 'Delayed' END")
)

In [ ]:
foo2.show()

In [ ]:
foo3 = foo2.drop("delay")
foo3.show()

In [ ]:
foo4 = foo3.withColumnRenamed("status", "flight_status")
foo4.show()

In [ ]:
pivot_before = spark.sql("""
SELECT * FROM (
  SELECT
    destination, 
    CAST(SUBSTRING(date, 0, 2) AS int) AS month,
    delay
  FROM departure_delays
  WHERE origin = 'SEA'
)
ORDER BY destination
""").show()

In [ ]:
pivot_after = spark.sql("""
SELECT * FROM (
  SELECT
    destination, 
    CAST(SUBSTRING(date, 0, 2) AS int) AS month,
    delay
  FROM departure_delays
  WHERE origin = 'SEA'
)
PIVOT ( 
  CAST(AVG(delay) AS DECIMAL(4, 2)) AS AvgDelay, 
  MAX(delay) AS MaxDelay
  FOR month in (1 JAN, 2 FEB)
)
ORDER BY destination
""").show()